In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
impressions_1 = pd.read_csv("./impressions_2022_02_04.csv")
#impressions_1 = impressions_1[impressions_1["gaze_valid"] == True]
impressions_2 = pd.read_csv("./impressions_2023_02_04.csv")
#impressions_2 = impressions_2[impressions_2["gaze_valid"] == True]
impressions_3 = pd.read_csv("./impressions_2021_02_04.csv")
impressions_4 = pd.read_csv("./impressions_2020_02_04.csv")


impressions = pd.concat([impressions_1, impressions_2, impressions_3, impressions_4])
#impressions = impressions[impressions["gaze_valid"] == True]

/tmp/ipykernel_20980/1331809118.py:9: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  impressions = pd.concat([impressions_1, impressions_2, impressions_3, impressions_4])


In [3]:
impressions["true_is_iab_inview"] = impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

In [4]:
def get_size(x):
    return f"{x['ad_width_chosen']}x{x['ad_height_chosen']}"

impressions["size"] = impressions.apply(lambda x: get_size(x), axis=1)

In [5]:
grouped_impressions = impressions.groupby(["part_year", "part_month", "hostname", "size", "chosen_brand"]).apply(
    lambda row: pd.Series(
        {
            "num_impressions": row["id"].count(),
            "num_fixations": row[row["is_fixated"] == True]["id"].count(),
            "total_fixation_duration": row["total_fixation_duration"].sum(),
            "num_inviews": row.loc[row["true_is_iab_inview"], "id"].count()
        }
    )
)
grouped_impressions = grouped_impressions.reset_index()
grouped_impressions["imp/fix_ratio"] = round(grouped_impressions["num_fixations"] / grouped_impressions["num_impressions"] * 100, 2)
grouped_impressions = grouped_impressions[grouped_impressions["num_impressions"] > 100]
grouped_impressions.sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
68035,2023,4,old.reddit.com,1180x240,leovegas,121.0,107.0,4010073.0,121.0,88.43
11944,2022,2,old.reddit.com,1180x240,leovegas,148.0,128.0,2055866.0,148.0,86.49
30284,2022,4,old.reddit.com,1180x240,leovegas,182.0,151.0,4300310.0,182.0,82.97
74266,2023,4,zkillboard.com,160x600,trocadero,131.0,77.0,29486.0,131.0,58.78
30288,2022,4,old.reddit.com,468x240,leovegas,155.0,90.0,639606.0,145.0,58.06
...,...,...,...,...,...,...,...,...,...,...
50596,2023,2,pricerunner.se,250x480,hallon,124.0,0.0,0.0,87.0,0.00
50671,2023,2,prisjakt.nu,180x150,elgiganten,371.0,0.0,0.0,0.0,0.00
62729,2023,4,google.com,125x230,samsung,344.0,0.0,0.0,45.0,0.00
50678,2023,2,prisjakt.nu,180x150,lendo,226.0,0.0,0.0,0.0,0.00


In [6]:
grouped_impressions["chosen_brand"].value_counts()

samsung          94
webhallen        52
netonnet         52
elgiganten       49
amazon           45
                 ..
chilimobil        1
centerpartiet     1
akavia            1
dollarstore       1
lidl              1
Name: chosen_brand, Length: 200, dtype: int64

In [7]:
other_grouped_impressions = impressions.groupby(["part_year", "part_month"]).apply(
    lambda row: pd.Series(
        {
            "num_impressions": row["id"].count(),
            "num_fixations": row[row["is_fixated"] == True]["id"].count(),
            "total_fixation_duration": row["total_fixation_duration"].sum(),
            "num_inviews": row.loc[row["true_is_iab_inview"], "id"].count()
        }
    )
)
other_grouped_impressions["imp/fix_ratio"] = round(other_grouped_impressions["num_fixations"] / other_grouped_impressions["num_impressions"] * 100, 2)
other_grouped_impressions

num_impressions  num_fixations  total_fixation_duration  \
part_year part_month                                                            
2020      2                 1791600.0            0.0                      0.0   
          4                  330171.0            0.0                      0.0   
2021      2                  212128.0            0.0                      0.0   
          4                  288971.0            0.0                      0.0   
2022      2                  318725.0        18265.0               36132553.0   
          4                  317797.0        18812.0               45153576.0   
2023      2                  343223.0        28100.0              138545786.0   
          4                  269358.0        29687.0              249887195.0   

                      num_inviews  imp/fix_ratio  
part_year part_month                              
2020      2                   0.0           0.00  
          4                   0.0           0.00  
2021      2                   0.0           0.00  
          4                   0.0           0.00  
2022      2              145838.0           5.73  
          4              129116.0           5.92  
2023      2              182651.0           8.19  
          4              135316.0          11.02

In [8]:
other_grouped_impressions["num_impressions"] = other_grouped_impressions["num_impressions"].astype("Int64")
other_grouped_impressions["num_fixations"] = other_grouped_impressions["num_fixations"].astype("Int64")
other_grouped_impressions["total_fixation_duration"] = other_grouped_impressions["total_fixation_duration"].astype("Int64")
other_grouped_impressions["num_inviews"] = other_grouped_impressions["num_inviews"].astype("Int64")
other_grouped_impressions

num_impressions  num_fixations  total_fixation_duration  \
part_year part_month                                                            
2020      2                   1791600              0                        0   
          4                    330171              0                        0   
2021      2                    212128              0                        0   
          4                    288971              0                        0   
2022      2                    318725          18265                 36132553   
          4                    317797          18812                 45153576   
2023      2                    343223          28100                138545786   
          4                    269358          29687                249887195   

                      num_inviews  imp/fix_ratio  
part_year part_month                              
2020      2                     0           0.00  
          4                     0           0.00  
2021      2                     0           0.00  
          4                     0           0.00  
2022      2                145838           5.73  
          4                129116           5.92  
2023      2                182651           8.19  
          4                135316          11.02

In [9]:
final_df = other_grouped_impressions.rename(
    columns={
        "part_year": "Year",
        "part_month": "Month",
        "num_impressions": "Impressions",
        "num_fixations": "Fixations",
        "num_inviews": "Inviews",
        "imp/fix_ratio": "Fixation Ratio"
    }
)
final_df.to_excel("covid_numbers.xlsx")

In [ ]:
2019 - 02 - 9964162
2019 - 04 - 9375309

2020 - 02 - 11449779
2020 - 04 - 21529207

2021 - 02 - 13012985
2021 - 04 - 15212634

2022 - 02 - 14005642
2022 - 04 - 13720203

2023 - 02 - 18318363
2023 - 04 - 23299611